**1 – Mount Drive & Load RA_COHORT_FINAL**

In [1]:
# CELL 1: Mount Google Drive and load the cleaned RA cohort

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

# Base path to your Scenario 2 folder
base_path = "/content/drive/MyDrive/Rahul_DTSC5082_Project/Scenario2_MIMIC"

# Path to the final RA cohort (change to .csv if needed)
ra_cohort_path = os.path.join(base_path, "outputs", "RA_COHORT_FINAL.parquet")

# Load the dataset
df = pd.read_parquet(ra_cohort_path)

# Quick sanity check
print(df.shape)
df.head()


Mounted at /content/drive
(15462, 44)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,lab_mean_CRP,lab_mean_Hematocrit,lab_mean_Hemoglobin,lab_mean_Platelets,lab_mean_WBC,lab_min_CRP,lab_min_Hematocrit,lab_min_Hemoglobin,lab_min_Platelets,lab_min_WBC
0,10002443,21329020,2183-10-17 23:20:00,2183-10-20 18:47:00,None,EW EMER.,P343TV,TRANSFER FROM HOSPITAL,HOME,Private,...,NaN,40.866667,13.466667,266.000000,16.533333,NaN,37.3,12.3,219.0,13.0
1,10003203,25146996,2153-04-26 02:05:00,2153-04-29 14:19:00,None,EU OBSERVATION,P57BOT,EMERGENCY ROOM,None,Medicare,...,NaN,26.150000,7.800000,307.000000,8.500000,NaN,23.3,7.6,296.0,8.4
2,10010718,29947356,2169-01-20 13:21:00,2169-01-27 14:20:00,None,OBSERVATION ADMIT,P50GUR,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,NaN,37.585714,12.285714,325.571429,10.014286,NaN,34.7,11.3,274.0,7.8
3,10010997,20783870,2139-04-28 16:45:00,2139-05-02 12:20:00,None,OBSERVATION ADMIT,P756E2,TRANSFER FROM HOSPITAL,HOME,Private,...,NaN,34.000000,11.200000,292.000000,6.300000,NaN,34.0,11.2,292.0,6.3
4,10010997,20783870,2139-04-28 16:45:00,2139-05-02 12:20:00,None,OBSERVATION ADMIT,P756E2,TRANSFER FROM HOSPITAL,HOME,Private,...,NaN,34.000000,11.200000,292.000000,6.300000,NaN,34.0,11.2,292.0,6.3


**2 — Create Flags + Basic Engineered Features**

In [4]:
# CELL 2 (Updated): Structured Feature Engineering - Flags + Ranges

import numpy as np
import pandas as pd

# O2 range using mean vs min (proxy for variability)
df['o2_range'] = df['o2_mean'] - df['o2_min']

# Flags based on established clinical cutoffs
df['fever_flag'] = (df['temp_max'] >= 38).astype(int)
df['tachy_flag'] = (df['hr_max'] >= 100).astype(int)
df['hypoxia_flag'] = (df['o2_min'] <= 92).astype(int)

# Pain severity category (ordinal)
def pain_category(x):
    if pd.isna(x):
        return np.nan
    if x <= 3:
        return 'mild'
    elif x <= 6:
        return 'moderate'
    else:
        return 'severe'

df['pain_category'] = df['pain_max'].apply(pain_category)

df[['temp_range', 'o2_range', 'fever_flag', 'tachy_flag', 'hypoxia_flag', 'pain_category']].head()


,temp_range,o2_range,fever_flag,tachy_flag,hypoxia_flag,pain_category
0,0.0,0.000000,1,0,0,mild
1,0.0,0.000000,1,0,0,NaN
2,0.8,1.100000,1,1,0,moderate
3,1.6,2.428571,1,0,0,severe
4,1.6,2.428571,1,0,0,severe


**3 — Add BP-based Hypotension Flag**

In [5]:
# CELL 3: Hypotension clinical flag (SBP < 90 or DBP < 60)

df['hypotension_flag'] = (
    (df['sbp_min'] <= 90) | (df['dbp_min'] <= 60)
).astype(int)

df[['sbp_min', 'dbp_min', 'hypotension_flag']].head()


,sbp_min,dbp_min,hypotension_flag
0,116.0,78.0,0
1,157.0,69.0,0
2,108.0,58.0,1
3,120.0,53.0,1
4,120.0,53.0,1


**4 — LAB ABNORMALITY FLAGS + CBC Summary Feature**

In [6]:
# CELL 4: Lab Feature Engineering - Clinical Flags + Ranges

import numpy as np

# ---- CRP (Inflammation marker) ----
# >10 mg/L is clinically significant inflammation
df['crp_high_flag'] = (df['lab_max_CRP'] > 10).astype(int)

# ---- WBC ----
# Normal range: 4 - 11 (x10^3/µL)
df['wbc_high_flag'] = (df['lab_max_WBC'] > 11).astype(int)
df['wbc_low_flag'] = (df['lab_min_WBC'] < 4).astype(int)

# ---- Hemoglobin (Anemia) ----
# ≤12 (female) or ≤13 (male) is anemia — but we don't have sex-specific yet
# Using general cutoff <12.0
df['hb_low_flag'] = (df['lab_min_Hemoglobin'] < 12).astype(int)

# ---- Platelets ----
# thrombocytopenia <150
df['platelet_low_flag'] = (df['lab_min_Platelets'] < 150).astype(int)

# ---- Hematocrit ----
# <36% (female) <41% (male), using general <36 cutoff
df['hct_low_flag'] = (df['lab_min_Hematocrit'] < 36).astype(int)

# ---- CBC abnormal count summary ----
df['cbc_abnormal_count'] = (
    df['wbc_high_flag'] +
    df['wbc_low_flag'] +
    df['hb_low_flag'] +
    df['platelet_low_flag'] +
    df['hct_low_flag']
)

# ---- Lab variability features ----
df['wbc_range'] = df['lab_max_WBC'] - df['lab_min_WBC']
df['hb_range'] = df['lab_max_Hemoglobin'] - df['lab_min_Hemoglobin']
df['platelet_range'] = df['lab_max_Platelets'] - df['lab_min_Platelets']
df['crp_range'] = df['lab_max_CRP'] - df['lab_min_CRP']

df[['crp_high_flag', 'wbc_high_flag', 'hb_low_flag',
    'platelet_low_flag', 'hct_low_flag', 'cbc_abnormal_count']].head()


,crp_high_flag,wbc_high_flag,hb_low_flag,platelet_low_flag,hct_low_flag,cbc_abnormal_count
0,0,1,0,0,0,1
1,0,0,1,0,1,2
2,0,1,1,0,1,3
3,0,0,1,0,1,2
4,0,0,1,0,1,2


**5 — Save the “Phase 2 structured cohort”**

In [7]:
# CELL 5: Save cohort with engineered structured features

output_path_struct = os.path.join(
    base_path,
    "outputs",
    "RA_COHORT_PHASE2_STRUCTURED.parquet"
)

df.to_parquet(output_path_struct, index=False)

print("Saved structured cohort to:")
print(output_path_struct)
print("Final shape:", df.shape)


Saved structured cohort to:
/content/drive/MyDrive/Rahul_DTSC5082_Project/Scenario2_MIMIC/outputs/RA_COHORT_PHASE2_STRUCTURED.parquet
Final shape: (15462, 63)


**6 — Load discharge notes**

In [8]:
# CELL 6: Load discharge notes from MIMIC-IV-Note

import pandas as pd
import os

notes_path = os.path.join(
    base_path,
    "data",
    "discharge.csv"
)

# Discharge.csv is large — use low_memory=False
notes_df = pd.read_csv(notes_path, low_memory=False)

print(notes_df.shape)
notes_df.head(3)


(331793, 8)


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...


**7 — Filter RA Notes**

In [9]:
# CELL 7: Filter discharge notes to RA patients/admissions

# Keep only needed columns from structured cohort
ra_ids = df[['subject_id', 'hadm_id']].drop_duplicates()

# Merge notes and RA cohort
ra_notes = notes_df.merge(
    ra_ids,
    on=['subject_id', 'hadm_id'],
    how='inner'
)

print("Total RA notes:", ra_notes.shape)
ra_notes.head(3)


Total RA notes: (4979, 8)


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10010997-DS-9,10010997,20783870,DS,9,2139-05-02 00:00:00,2139-05-07 15:37:00,\nName: ___ Unit No: ___\n \...
1,10021930-DS-13,10021930,20480646,DS,13,2177-01-14 00:00:00,2177-01-15 07:20:00,\nName: ___ Unit No: ___\...
2,10028125-DS-14,10028125,29060034,DS,14,2171-07-01 00:00:00,2171-07-02 10:06:00,\nName: ___ Unit No: ___\n \n...


**8 — Text Cleaning Function**

In [10]:
# CELL 8: Clean discharge note text

import re

def clean_note_text(t):
    if pd.isna(t):
        return ""

    # Remove de-identified tokens like [**Doctor Last Name**]
    t = re.sub(r"\[\*\*.*?\*\*\]", " ", t)

    # Remove template lines such as "Name: ___", "Unit No: ___"
    t = re.sub(r"Name:\s*_*.*", " ", t)
    t = re.sub(r"Unit\s*No:\s*_*.*", " ", t)

    # Remove excessive underscores (_____ etc.)
    t = re.sub(r"_+", " ", t)

    # Remove multiple spaces
    t = re.sub(r"\s+", " ", t)

    # Strip leading/trailing whitespace
    t = t.strip()

    return t.lower()  # lowercase for transformer consistency

# Apply cleaning
ra_notes['clean_text'] = ra_notes['text'].apply(clean_note_text)

# Show example
ra_notes[['text', 'clean_text']].head(3)


,text,clean_text
0,\nName: ___ Unit No: ___\n \...,admission date: discharge date: date of birth:...
1,\nName: ___ Unit No: ___\...,admission date: discharge date: date of birth:...
2,\nName: ___ Unit No: ___\n \n...,admission date: discharge date: date of birth:...


In [11]:
# CELL 8B: Show longer excerpts from cleaned notes

for i in range(3):
    print("----- NOTE", i, "-----")
    print(ra_notes['clean_text'].iloc[i][:1000])  # show first 1000 chars
    print("\n")


----- NOTE 0 -----
admission date: discharge date: date of birth: sex: f service: orthopaedics allergies: no known allergies / adverse drug reactions attending: . chief complaint: r if pain major surgical or invasive procedure: procedure: irrigation, washout and debridement right index finger distal interphalangeal joint. history of present illness: year-old right-hand dominant nurse at with severe ra on methotrexate who underwent excision of distal right if mass just proximal to eponychial fold concerning for mucus cyst on at . she noted some drainage from the incision starting in the past , she had worsening pain therefore went to earlier today where they cultured purulent discharge and gpcs in clusters and pairs were observed on gram stain, gave vancomycin and zosyn, and was superficially washed out and digital block performed for pain control. she was transferred to for further management. she denies fevers or chills, only increasing pain, drainage, and swelling of the digit. past 

**9 — Keep only non-empty notes (safety before embeddings)**

In [12]:
# CELL 9: Filter out empty / tiny notes before embedding

# Keep notes with at least 20 characters of cleaned text
min_len = 20

initial_n = ra_notes.shape[0]
ra_notes = ra_notes[ra_notes['clean_text'].str.len() >= min_len].copy()
ra_notes = ra_notes.reset_index(drop=True)

print(f"Kept {ra_notes.shape[0]} notes out of {initial_n} after length filter.")
ra_notes[['note_id', 'hadm_id', 'clean_text']].head(3)


Kept 4979 notes out of 4979 after length filter.


,note_id,hadm_id,clean_text
0,10010997-DS-9,20783870,admission date: discharge date: date of birth:...
1,10021930-DS-13,20480646,admission date: discharge date: date of birth:...
2,10028125-DS-14,29060034,admission date: discharge date: date of birth:...


In [13]:
# FINAL CELL in Notebook 1: Save cleaned RA notes

clean_notes_path = os.path.join(
    base_path,
    "outputs",
    "RA_NOTES_CLEANED.parquet"
)

ra_notes.to_parquet(clean_notes_path, index=False)

print("Saved cleaned RA notes to:")
print(clean_notes_path)
print("Shape:", ra_notes.shape)


Saved cleaned RA notes to:
/content/drive/MyDrive/Rahul_DTSC5082_Project/Scenario2_MIMIC/outputs/RA_NOTES_CLEANED.parquet
Shape: (4979, 9)
